In [3]:
import csv
import requests
import re
import time
import itertools
import pandas as pd
import hgvs
import hgvs.parser
import hgvs.dataproviders.uta
import hgvs.assemblymapper

In [4]:
#Gets a protein sequence in FASTA format given the ensembl transcript ID
def get_protein_sequence(canonical_transcript):
    server = "http://grch37.rest.ensembl.org"
    ext = "/sequence/id/" + canonical_transcript + "?content-type=text/x-fasta;type=protein"
    r = requests.get(server + ext, headers={"Content-Type": "text/x-fasta"})
    time.sleep(3)
    
    if not r.ok:
        try:
            r.raise_for_status()
            return "error"
        except requests.exceptions.HTTPError: #I should catch the error too or print, to find out which specific genes
            pass
    seqlist = r.text.split("\n", 1)
    
    if len(seqlist) == 2: #A lot of these transcripts are noncoding or introns, don't know why
        sequence = seqlist[1]
        sequence = sequence.replace("\n", "")
        return sequence
    #print("Sequence found")

#Adapted from https://github.com/xjenny2/phospho-programs/blob/master/ensembl.py

In [3]:
tsv_file = open("filtered_mapRef.tsv")
mart_tsv = csv.reader(tsv_file, delimiter="\t", quotechar='"')
df = pd.DataFrame(mart_tsv)

In [4]:
dfRef = df[df[6].astype(bool)] #filters tsv file such that it only considers all with a RefSeq Transcript

In [5]:
df2 = pd.read_csv('clinvar.csv', index_col = 0)
dfClinvar = pd.DataFrame(df2)

In [6]:
column = ['CHROM', 'POS', 'REF', 'ALT', 'MC', 'CLNSIG', 'NC', 'NM','NP','ENST', 'ProteinSeq', 'Distance to Next Methionine', 'Percentage Retained']
dfMap = pd.DataFrame(columns = column) #initializes dataframe with column headings

In [7]:
hdp = hgvs.dataproviders.uta.connect()
am = hgvs.assemblymapper.AssemblyMapper(hdp,assembly_name = 'GRCh38', alt_aln_method = "splign", replace_reference = True)

In [9]:
dfClinvar.shape

(31008, 10)

In [10]:
#to do add a check for '>' at position -2
data = [] #Fills in the dataframe with clinvar file information and converts gene to transcript to protein
for index, row in dfClinvar.loc[7779:].iterrows():
    if str(row[8]).startswith('NC') and row[8][-2] == '>':
        hgvs_g = str(row[8])
        hp = hgvs.parser.Parser()
        var_g = hp.parse_hgvs_variant(hgvs_g) #parses HGVS
        transcripts = am.relevant_transcripts(var_g)
        if transcripts[0].startswith('NR'):
            continue
        else:
            var_c = am.g_to_c(var_g, transcripts[0]) #converts Gene to Transcript
            var_p = am.c_to_p(var_c) #converts Transcript to Protein
            time.sleep(2)
    
    values = []
    values.append(row[0])
    values.append(row[1])
    values.append(row[3])
    values.append(row[4])
    values.append(row[7])
    values.append(row[9])
    values.append(row[8])
    values.append(str(var_c))
    values.append(str(var_p))
    
    zipped = zip(column, values)
    dictionary = dict(zipped)
    data.append(dictionary)

Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_198576.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001029885.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001123377.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Fai

Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_000098.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_176782.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_152268.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_174936.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001004303.4&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegro

Failure 0/3; retry in 1 seconds
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_000396.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Normalization of intronic variants is not supported; returning unnormalized variant
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_002796.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_015100.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutil

Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_005807.4&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001287820.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001312672.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_030787.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001206846.1&rettype=fasta&tool=bioutils&email=biocommons-dev@goo

Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001100.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_004481.4&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_022051.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001164545.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001077397.1&rettype=fasta&tool=bioutils&email=biocommons-dev@google

...

Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001059.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Normalization of intronic variants is not supported; returning unnormalized variant
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_006903.4&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001136257.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_183075.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=

Failure 0/3; retry in 1 seconds
Normalization of intronic variants is not supported; returning unnormalized variant
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_004237.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001003841.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Normalization of intronic variants is not supported; returning unnormalized variant
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_138809.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001288715.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 

Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000005.10&rettype=fasta&seq_start=71592977&seq_stop=71592977&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000005.10&rettype=fasta&seq_start=71599757&seq_stop=71599757&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_022132.4&rettype=fasta&seq_start=700&seq_stop=720&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000005.10&rettype=fasta&seq_start=72194949&seq_stop=72194949&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https:/

Failure 0/3; retry in 1 seconds
Failure 1/3; retry in 2 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_000038.5&rettype=fasta&seq_start=1319&seq_stop=1319&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_000038.5&rettype=fasta&seq_start=1336&seq_stop=1336&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000005.10&rettype=fasta&seq_start=112819294&seq_stop=112819294&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_000038.5&rettype=fasta&seq_start=1347&seq_stop=1367&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Faile

HGVSDataNotAvailableError: Failed to fetch NC_000005.10 from bioutils.seqfetcher (network fetching) (Failed to fetch NC_000005.10 (502 Server Error: Bad Gateway for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000005.10&rettype=fasta&seq_start=112827230&seq_stop=112827230&tool=bioutils&email=biocommons-dev@googlegroups.com))

In [11]:
dfMap = dfMap.append(data, True) 
dfMap['ENST'] = dfMap['ENST'].fillna('0')

/Users/dylantom/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [ ]:
for index, row in dfMap.iterrows():
    NM = row[7].split('.')[0] #obtains the initial transcript
    for x, y in dfRef.iterrows():
        Match = y[6].split('.')[0] 
        if NM == Match: #checks a map for a transcript
            dfMap.loc[index, 'ENST'] = y[4]

In [5]:
dfMap = pd.read_csv('results_1_temp.csv', index_col = 0)

In [6]:
dfMap = pd.DataFrame(dfMap)

In [7]:
dfMap.shape

(8370, 13)

In [ ]:
dfMap.head()

In [8]:
for index, row in dfMap.iterrows():
    if row[9] != 0:
        dfMap.loc[index, 'ProteinSeq'] = get_protein_sequence(row[9])

ReadTimeout: HTTPConnectionPool(host='grch37.rest.ensembl.org', port=80): Read timed out. (read timeout=None)

In [10]:
dfMap.head()

,CHROM,POS,REF,ALT,MC,CLNSIG,NC,NM,NP,ENST,ProteinSeq,Distance to Next Methionine,Percentage Retained
0,1,943995,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.943995C>T,NM_152486.2:c.1888C>T,NP_689699.2:p.(Arg630Ter),0,NaN,NaN,NaN
1,1,944051,T,G,SO:0001587|nonsense,Uncertain_significance,NC_000001.11:g.944051T>G,NM_152486.2:c.1944T>G,NP_689699.2:p.(Tyr648Ter),0,NaN,NaN,NaN
2,1,1014143,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1014143C>T,NM_005101.3:c.163C>T,NP_005092.1:p.(Gln55Ter),ENST00000649529,NaN,NaN,NaN
3,1,1014359,G,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1014359G>T,NM_005101.3:c.379G>T,NP_005092.1:p.(Glu127Ter),ENST00000649529,NaN,NaN,NaN
4,1,1041582,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1041582C>T,NM_198576.3:c.1057C>T,NP_940978.2:p.(Gln353Ter),ENST00000379370,MAGRSHPGPLRPLLPLLVVAACVLPGAGGTCPERALERREEEANVV...,NaN,NaN


In [13]:
for index, row in dfMap.iterrows():
    Protein_Mutation_Position = int(re.findall(r'\d+', row[8])[-1])
    if row[10] != pd.isna():
        #Index and position are difference; Index = Position - 1
        sub_seq = row[10][Protein_Mutation_Position - 1:-1]
        distance = 0
        retained = 0
        for metIndex, aminoAcid in enumerate(sub_seq):
            if aminoAcid == 'M':
                distance = metIndex
                remain = sub_seq[metIndex:-1]
                retained = len(remain)/len(row[10])
                break
        dfMap.loc[index, 'Distance to Next Methionine'] = distance
        dfMap.loc[index, 'Percentage Remtained'] = retained

TypeError: isna() missing 1 required positional argument: 'obj'

In [ ]:
dfMap = dfMap[dfMap['ProteinSeq'].notna()]

In [ ]:
dfMap.shape

In [ ]:
dfMap.to_csv('results_1_temp.csv')